<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_5_mixed_pubmed_cadec_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heel pain
2,5,increase my heart rate
3,5,tachycardia
4,8,severe arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-5.test.csv",header=None)
test.head()

,0,1
0,0,fear
1,0,scared to death
2,2,heel pain
3,2,heels hurt
4,2,pain in my right heel


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_cadec_askpatient_mixed.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'muscle',
 'a',
 '.',
 'was',
 'with',
 'i',
 ',',
 'diclofenac',
 'my',
 'for',
 'were',
 'severe',
 'on',
 'lower',
 'myalgia',
 'fatigue',
 'loss',
 'is',
 'that',
 'weakness',
 '-',
 'cramp',
 'by',
 'sodium',
 'limb',
 'back',
 'stomach',
 'at',
 'drug',
 'as',
 'this',
 'depression',
 'not',
 'it',
 'or',
 'cramps',
 'mg',
 'patients',
 'memory',
 'joint',
 'upper',
 'gas',
 'leg',
 'from',
 'legs',
 'all',
 'after',
 'pains',
 'knee',
 'have',
 'headache',
 'shoulder',
 'unable',
 'group',
 'neck',
 '/',
 'be',
 'gastrointestinal',
 'foot',
 'treatment',
 'arthralgia',
 'excessive',
 'study',
 'aches',
 'arthritis',
 'like',
 'no',
 'hip',
 'cholesterol',
 'an',
 'effects',
 'arthrotec',
 'had',
 'numbness',
 'voltaren',
 'insomnia',
 'time',
 'disease',
 'walk',
 'but',
 'both',
 'feeling',
 'heart',
 'problems',
 'bleeding',
 'p',

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.805445,3.474133,0.434762,33:39


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.369097,3.249114,0.455380,33:24


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.202913,3.220853,0.458801,33:35


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.162460,3.199973,0.461006,33:33


In [25]:
learn.save_encoder('pubmed_cadec_askpatient_mixed_fold6_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_askpatient_mixed_fold6_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.661890,3.373852,0.416347,01:05


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.627909,2.791506,0.487867,01:17


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.129025,2.358850,0.572158,03:30


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.809603,2.107027,0.619413,03:07
1,1.545039,1.969225,0.656450,03:17


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.442569,1.787132,0.673052,03:22
1,1.207005,1.700711,0.704981,03:16


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.128903,1.580038,0.716475,03:34
1,0.937473,1.525526,0.727969,03:11


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.908986,1.454283,0.742018,03:34
1,0.818732,1.401490,0.756066,03:24


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.700545,1.384297,0.753512,03:10
1,0.649320,1.303949,0.768838,03:21


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.593632,1.295395,0.765006,03:22
1,0.564124,1.283410,0.780332,03:28
2,0.500971,1.196322,0.789272,03:26
3,0.422072,1.208660,0.787995,03:20


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.417425,1.238680,0.784163,03:35
1,0.449298,1.137349,0.803321,03:18
2,0.378401,1.154473,0.810983,03:19
3,0.319754,1.138131,0.816092,03:23


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.346460,1.187371,0.802043,03:26
1,0.387220,1.201870,0.802043,03:14
2,0.316256,1.156366,0.807152,03:27
3,0.260685,1.168515,0.814815,03:18


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.270278,1.165576,0.818646,03:14
1,0.311482,1.146479,0.819923,03:23
2,0.268636,1.168171,0.812261,03:19
3,0.238539,1.199411,0.817369,03:29


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.217802,1.205012,0.825032,03:34
1,0.269841,1.166451,0.819923,03:33
2,0.236397,1.148585,0.825032,03:32
3,0.212125,1.206503,0.825032,03:13


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.203561,1.289209,0.821200,03:14
1,0.215008,1.252764,0.826309,03:25
2,0.201617,1.229879,0.825032,03:25
3,0.183503,1.192889,0.825032,03:26


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.172001,1.223193,0.821200,03:18
1,0.214624,1.246507,0.816092,03:19
2,0.188498,1.163401,0.837803,03:16
3,0.169468,1.187900,0.837803,03:25


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.190936,1.283339,0.831418,03:19
1,0.187066,1.206137,0.825032,03:22


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.187733,1.290418,0.816092,03:21
1,0.186411,1.324522,0.823755,03:18


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.197682,1.169919,0.835249,03:23


In [44]:
# save the best model

learn.save_encoder('pubmed_cadec_askpatient_mixed_fold5')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[0, 0, 2, 2, 289, 8, 8, 8, 8, 16, 16, 16, 71, 27, 27, 27, 110, 34, 34, 209, 37, 37, 37, 37, 37, 37, 315, 48, 49, 79, 49, 49, 49, 49, 187, 53, 53, 58, 323, 853, 954, 71, 71, 71, 71, 129, 209, 265, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 80, 80, 220, 83, 733, 585, 92, 94, 94, 94, 94, 94, 94, 95, 96, 96, 96, 98, 100, 100, 100, 100, 100, 100, 100, 100, 100, 103, 104, 712, 107, 107, 108, 108, 108, 111, 730, 111, 111, 111, 111, 111, 182, 111, 122, 119, 122, 472, 122, 122, 125, 839, 125, 125, 125, 125, 125, 125, 135, 1026, 137, 137, 344, 148, 588, 151, 157, 865, 509, 509, 166, 166, 166, 171, 172, 172, 289, 289, 172, 172, 172, 172, 172, 172, 214, 173, 214, 620, 122, 165, 181, 181, 182, 182, 183, 183, 183, 183, 183, 183, 183, 183, 184, 240, 240, 190, 190, 190, 289, 289, 188, 190, 190, 190, 190, 190, 717, 855, 193, 196, 197, 197, 197, 197, 197, 202, 202, 205, 209, 210, 210, 211

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

675
0.7794457274826789
